In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import markdown

### real estate data

In [ ]:
properties_1 = pd.read_excel('untidy_Montreal_property_listings.xlsx')
properties_1.head()

price                                 region  \
0  $849,000      Mercier / Hochelaga / Maisonneuve   
1  $420,000  Villeray / St-Michel / Parc-Extension   
2  $810,000  Villeray / St-Michel / Parc-Extension   
3       NaN                                    NaN   
4  $699,000     Pointe-Aux-Trembles / Montréal-Est   

                                  address                 bedrooms  \
0                5185-5187 RUE DESMARTEAU  Bedrooms\n            3   
1                 417-8635 RUE LAJEUNESSE  Bedrooms\n            2   
2  8517-A-8515-8517 avenue de l'esplanade  Bedrooms\n            8   
3                                     NaN                      NaN   
4                          11901 Victoria  Bedrooms\n            3   

                               bathrooms  \
0  Bathrooms + Half baths\n            1   
1  Bathrooms + Half baths\n            1   
2  Bathrooms + Half baths\n            3   
3                                    NaN   
4  Bathrooms + Half baths\n            1   

                                    living area lot_dimension   Latitude  \
0                     Lot dimensions\n2,176 ft²           NaN  45.609226   
1  Living space area (basement exclu.)\n957 ft²           NaN  45.546245   
2                     Lot dimensions\n2,850 ft²           NaN  45.538018   
3                                           NaN           NaN  45.515334   
4                     Lot dimensions\n3,060 ft²           NaN  45.638808   

   Longitude  
0 -73.533193  
1 -73.636030  
2 -73.639644  
3 -73.682818  
4 -73.493559

#### Remove nulls in the address fields

In [ ]:
properties_1.isna().sum()

price             78
region            78
address           83
bedrooms          78
bathrooms         95
living area      131
lot_dimension    733
Latitude           0
Longitude          0
dtype: int64

In [ ]:
#drop where there are nulls in the address column
properties = properties_1.copy()
properties.dropna(subset=['address'], inplace=True)

### Data integration
***
    Add income, population and number of households to the main dataset from census data:
        
        The properties datset have value fields: address without postal code and region
        
        The census data from Statistique Canada have first three characters of the postal code
***
    To integrate the data, we must resolve that the address entity of the properties dataset has a postal code
    Then we can extract the first three characters of the postal code to link to the census data
    
Using **Geocoders from Awesome Table** we were able to get longitude and latitude for the properties.
After we got the coordinates, we can find the postal code with Nominatim from geopy geocoders package

#### Get postal code of address with Nominatim from Geopy Geocoders

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
geolocator = Nominatim(user_agent = "check_1")

def get_zip_code(x):
    location = geolocator.reverse("{}, {}".format(x['Latitude'],x['Longitude']))
    return location.raw['address']['postcode']
properties['zipcode'] = properties.apply(lambda x: get_zip_code(x), axis = 1)

### Export as csv file

In [ ]:
properties.head()

price                                 region  \
0  $849,000      Mercier / Hochelaga / Maisonneuve   
1  $420,000  Villeray / St-Michel / Parc-Extension   
2  $810,000  Villeray / St-Michel / Parc-Extension   
4  $699,000     Pointe-Aux-Trembles / Montréal-Est   
5  $349,000            Rosemont / La Petite Patrie   

                                  address                 bedrooms  \
0                5185-5187 RUE DESMARTEAU  Bedrooms\n            3   
1                 417-8635 RUE LAJEUNESSE  Bedrooms\n            2   
2  8517-A-8515-8517 avenue de l'esplanade  Bedrooms\n            8   
4                          11901 Victoria  Bedrooms\n            3   
5                         5919 rue Chabot  Bedrooms\n            1   

                               bathrooms  \
0  Bathrooms + Half baths\n            1   
1  Bathrooms + Half baths\n            1   
2  Bathrooms + Half baths\n            3   
4  Bathrooms + Half baths\n            1   
5  Bathrooms + Half baths\n            1   

                                    living area lot_dimension   Latitude  \
0                     Lot dimensions\n2,176 ft²           NaN  45.609226   
1  Living space area (basement exclu.)\n957 ft²           NaN  45.546245   
2                     Lot dimensions\n2,850 ft²           NaN  45.538018   
4                     Lot dimensions\n3,060 ft²           NaN  45.638808   
5  Living space area (basement exclu.)\n537 ft²           NaN  45.542159   

   Longitude  zipcode  
0 -73.533193  H1L 1C9  
1 -73.636030  H2P 0B5  
2 -73.639644  H2N 1M4  
4 -73.493559  H1B 1P2  
5 -73.590741  H2G 1S9

In [ ]:
properties.to_csv('properties_with_postal_code.csv', index=False)